### Imports

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

### File Name

In [2]:
data_file_name = "Lohmans SupplementaryData Ligation observation tables.xlsx"

### Data Generation

In [56]:
# INPUT: File name (should be in the same directory as this file)
# OUTPUT: a pandas df file that can access intensities via df["AAA"]["TTT"]
def get_data_matrix(data_file_name): 
    dir_path = os.path.dirname(os.path.realpath("Data Workflow"))
    # pandas dataframe [this is the matrix right from xlsx file]
    df = pd.read_excel(dir_path + "/" + data_file_name)
    # this is changing the index to TTT or AAA as we would like
    df.set_index(df.columns[0], inplace = True)
    return df

# INPUT: A pandas data matrix
# OUTPUT: A numpy array of the form [("AAA", "TTT", [intensity value])]
def get_data_tuple(data_matrix):
    output = []
    for pair_1 in data_matrix.columns: 
        for pair_2 in data_matrix.index:
            tup = (pair_1, pair_2, data_matrix[pair_1][pair_2])
            output.append(tup)
    return np.array(output)

#INPUT: a tuple array of the data
#OUTPUT: a normalized tuple array of the data (helps with comparision)
def normalized_data_tuple(data):
    norm_data = data.copy()
    max_value = max([int(pair[2]) for pair in data])
    for pair in norm_data:
        pair[2] = int(pair[2])/max_value
    return norm_data

In [57]:
# getting data from the file to a matrix
matrix = get_data_matrix(data_file_name)
# flattinging it into the form e.g. ("AAA", "TTT", 5)
tuples = get_data_tuple(matrix)
# formalizing the outputs
x = normalized_data_tuple(tuples)

In [73]:
def encoder(overhang): 
    my_dict = {"A": '10', "T": '01', "G": '10', "C":'00'}
    binary_sequence = ""
    for base in overhang:
        binary_sequence += my_dict[base]
    return binary_sequence

def encoding_data(data):
    new_data = []
    for elem1, elem2, num in data:
        new_data.append([encoder(elem1 + elem2), float(num)])
    return new_data

### Linear Regression Model

In [70]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression


'0000010010'

In [77]:
data = encoding_data(x)

import numpy as np
from sklearn.linear_model import LinearRegression

# Define a function to convert a string into a vector of binary features
def string_to_binary_features(s):
    return np.array(list(s)).astype(int)

# Extract the features and target variable
X = np.array([string_to_binary_features(d[0]) for d in data])
y = np.array([d[1] for d in data])

print(X)

# Fit the linear regression model
model = LinearRegression()
model.fit(X, y)

# Make a prediction for a new input string
new_data = [["0110010001"]]
X_new = np.array([string_to_binary_features(d[0]) for d in new_data])
y_pred = model.predict(X_new)

print(y_pred)  # predicted output for the new input


[[1 0 1 ... 1 0 1]
 [1 0 1 ... 1 0 1]
 [1 0 1 ... 1 0 1]
 ...
 [0 1 0 ... 0 1 0]
 [0 1 0 ... 0 1 0]
 [0 1 0 ... 0 1 0]]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 12 is different from 10)